# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import scipy as sc
from scipy import signal

# Data Preprocessing

## Raw Dataset

In [ ]:
df = pd.read_csv("/content/230_Users_Acceleromter_Data.csv")
df

,Unnamed: 0,X,Y,Z,Label
0,0,0.340509,8.308413,4.140585,0.0
1,1,0.381370,8.390134,4.249548,0.0
2,2,0.272407,8.471856,4.018002,0.0
3,3,0.149824,8.430995,4.290409,0.0
4,4,0.272407,8.430995,4.481094,0.0
...,...,...,...,...,...
3449995,3449995,6.360573,1.495091,7.165300,229.0
3449996,3449996,6.322253,1.456771,7.395222,229.0
3449997,3449997,6.322253,1.801653,7.280261,229.0
3449998,3449998,6.437214,1.993255,7.203620,229.0


In [ ]:
df = df.drop("Unnamed: 0",axis=1)
df

,X,Y,Z,Label
0,0.340509,8.308413,4.140585,0.0
1,0.381370,8.390134,4.249548,0.0
2,0.272407,8.471856,4.018002,0.0
3,0.149824,8.430995,4.290409,0.0
4,0.272407,8.430995,4.481094,0.0
...,...,...,...,...
3449995,6.360573,1.495091,7.165300,229.0
3449996,6.322253,1.456771,7.395222,229.0
3449997,6.322253,1.801653,7.280261,229.0
3449998,6.437214,1.993255,7.203620,229.0


## Distributing User Data to Array

In [ ]:
length = 15000
N_Users=175

In [ ]:
import time
start_time = time.time()
k=0
X=[0 for p in range(N_Users)] # Final Array
for i in range(0,length*N_Users,length):
    temp=[]
    for j in range(length):
        x_y_z = [df.iloc[i+j,0],df.iloc[i+j,1],df.iloc[i+j,2]]
        temp.append(x_y_z)
    X[k]=temp
    k+=1 #incrementing [Final Array] indices
print("--- %s seconds ---" % (time.time() - start_time))

--- 187.54916834831238 seconds ---


In [ ]:
t=np.array(X)
t.shape

(175, 15000, 3)

## Algo for creating windows

In [ ]:
# Main Algo for Creating Windows .....#1

def create(df,wind_size,hop_size,m_index):
  #l=((len(df.x)-wind_size)/hop_size)+1
  #rows, cols = (wind_size, 3) 
#  a = [[0 for i in range(cols)] for j in range(rows)] 
  array= [0 for i in range(int((length-wind_size)/hop_size)+1)]
  k = 0
  for i in range(0,(length-wind_size+1),hop_size):
      a=create1(df,i,wind_size,m_index)
       #array.append(a)
      array[k] = a
      k+=1
  #return pd.DataFrame(array)
  return array


In [ ]:
# Main Algo for Creating Windows .....#2

def create1(df,x,w,m_index):
     rows = w 
     cols = 3
     arr = [[0 for i in range(cols)] for j in range(rows)] 
     for i in range(0,w,1):
        arr[i][0]=df[x+i][0]
        arr[i][1]=df[x+i][1]
        arr[i][2]=df[x+i][2]
     arr = np.array(arr)
     f1,t1,x = signal.stft(arr[:,0],5,nperseg=10)
     f2,t2,y = signal.stft(arr[:,1],5,nperseg=10)
     f3,t3,z = signal.stft(arr[:,2],5,nperseg=10)
     temp_a = np.concatenate((abs(np.array(x)),abs(np.array(y)),abs(np.array(z))),axis=0)
     return temp_a.flatten()

## Final Array for training

In [ ]:
import time
start_time = time.time()
X_final = [0 for i in range(N_Users)]
for m_index in range(N_Users):
  X_final[m_index]=create(X[m_index],225,10,m_index)
print("--- %s seconds ---" % (time.time() - start_time))

--- 281.38446712493896 seconds ---


In [ ]:
t=np.array(X_final[0])
t.shape             # The shape gives(number of windows,shape of TF plot image)

(1478, 828)

In [ ]:
X_input=[]
for i in range(N_Users):
  temp = X_final[i]
  X_input += temp
X_input = np.array(X_input)

## Creating Labels

In [ ]:
y = [0 for i in range(X_input.shape[0])]
for i in range(X_input.shape[0]):
  y[i]=int(i*N_Users/X_input.shape[0])
y = np.array(y)

## Final Input

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_input,y,test_size = 0.2,shuffle=True)

# BASELINES

## ML Classifiers

In [ ]:
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import neighbors
from sklearn import metrics
from sklearn import neural_network

In [ ]:
clf_Logistic_Reg = sklearn.linear_model.LogisticRegression(max_iter=800)
clf_dt = sklearn.tree.DecisionTreeClassifier()
clf_rf = sklearn.ensemble.RandomForestClassifier()
clf_knn = sklearn.neighbors.KNeighborsClassifier()
clf_nn = sklearn.neural_network.MLPClassifier()

## Performance Metrics

### Logistic Regression

In [ ]:
start_time = time.time()
clf_Logistic_Reg.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2119.0776693820953 seconds ---


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
start_time = time.time()
y_pred_train_Log = clf_Logistic_Reg.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.0506551265716553 seconds ---


In [ ]:
start_time = time.time()
y_pred_test_Log = clf_Logistic_Reg.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.2786436080932617 seconds ---


In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_Log)
acc_test = metrics.accuracy_score(y_test,y_pred_test_Log)
print('Logistic Regression:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Logistic Regression:
Accuracy: 0.3252 val_acc: 0.2453


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_Log, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.0758 Precision: 0.0926 F-Measure: 0.0833


### SVM

In [ ]:
#start_time = time.time()
#y_pred_train_svm = clf_SVM.predict(X_train)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#start_time = time.time()
#y_pred_test_svm = clf_SVM.predict(X_test)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#acc_train = metrics.accuracy_score(y_train,y_pred_train_svm)
#acc_test = metrics.accuracy_score(y_test,y_pred_test_svm)
#print('SVM:')
#print('Accuracy: %.4f' % acc_train, 'val_acc: %.4f' %acc_test)

In [ ]:
#recall = metrics.recall_score(y_test, y_pred_test_svm, labels=[1,2], average='micro')
#precision = metrics.precision_score(y_test,y_pred_test_svm, labels=[1,2], average='micro')
#f1 = metrics.f1_score(y_test, y_pred_test_svm, labels=[1,2], average='micro')
#print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

### Decision Tree

In [ ]:
start_time = time.time()
clf_dt.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2075.8634638786316 seconds ---


In [ ]:
start_time = time.time()
y_pred_train_dt = clf_dt.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.7243244647979736 seconds ---


In [ ]:
start_time = time.time()
y_pred_test_dt = clf_dt.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.4331040382385254 seconds ---


In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_dt)
acc_test = metrics.accuracy_score(y_test,y_pred_test_dt)
print('Decision Tree:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

Decision Tree:
Accuracy: 1.0000 val_acc: 0.2655


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_dt, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_dt, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_dt, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.0943 Precision: 0.0924 F-Measure: 0.0934


### Random Forest

In [ ]:
start_time = time.time()
clf_rf.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
y_pred_train_rf = clf_rf.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
y_pred_test_rf = clf_rf.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_rf)
acc_test = metrics.accuracy_score(y_test,y_pred_test_rf)
print('Random Forest:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_rf, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_rf, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

### KNN

In [ ]:
start_time = time.time()
clf_knn.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 32.76158356666565 seconds ---


In [ ]:
#start_time = time.time()
#y_pred_train_knn = clf_knn.predict(X_train)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
y_pred_test_knn = clf_knn.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 6052.137808084488 seconds ---


In [ ]:
#acc_train = metrics.accuracy_score(y_train,y_pred_train_knn)
acc_test = metrics.accuracy_score(y_test,y_pred_test_knn)
print('KNN:')
#print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)
print('val_acc: %.4f' %acc_test)

KNN:
val_acc: 0.4055


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_knn, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_knn, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test,y_pred_test_knn, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.2576 Precision: 0.1485 F-Measure: 0.1884


### MLP

In [ ]:
start_time = time.time()
clf_nn.fit(X_train,y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1670.9286003112793 seconds ---


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
start_time = time.time()
y_pred_train_nn = clf_nn.predict(X_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.212235927581787 seconds ---


In [ ]:
start_time = time.time()
y_pred_test_nn = clf_nn.predict(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.719841480255127 seconds ---


In [ ]:
acc_train = metrics.accuracy_score(y_train,y_pred_train_nn)
acc_test = metrics.accuracy_score(y_test,y_pred_test_nn)
print('MLP:')
print('Accuracy: %.4f' % acc_train,'val_acc: %.4f' %acc_test)

MLP:
Accuracy: 0.4454 val_acc: 0.4141


In [ ]:
recall = metrics.recall_score(y_test, y_pred_test_nn, labels=[1,2], average='micro')
precision = metrics.precision_score(y_test,y_pred_test_nn, labels=[1,2], average='micro')
f1 = metrics.f1_score(y_test, y_pred_test_nn, labels=[1,2], average='micro')
print('Recall: %.4f' % recall,'Precision: %.4f' % precision,'F-Measure: %.4f' % f1)

Recall: 0.3333 Precision: 0.1523 F-Measure: 0.2091
